In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv("../data/degree.csv")
print df.shape
df.drop(df.columns[-1], axis=1, inplace=True)
print df.head()

(126073, 3)
                       degree    count
0                          BS  94317.0
1                          BA  84842.0
2     Bachelor of Arts (B.A.)  77008.0
3  Bachelor of Science (B.S.)  69426.0
4    Bachelor of Science (BS)  66456.0


In [3]:
def is_complex_row(x):
    if isinstance(x, str) and x.count('(') == x.count(')') == 1 and x[-1] == ')':
        return True
    return False

In [4]:
is_complex = df['degree'].apply(is_complex_row)
is_complex.head()

0    False
1    False
2     True
3     True
4     True
Name: degree, dtype: bool

In [5]:
df_complex = df[is_complex]
df_complex.head()

,degree,count
2,Bachelor of Arts (B.A.),77008.0
3,Bachelor of Science (B.S.),69426.0
4,Bachelor of Science (BS),66456.0
12,Bachelor of Arts (BA),28212.0
15,Bachelor of Business Administration (B.B.A.),19819.0


In [6]:
df_complex.loc[:, 'degree'] = df_complex.loc[:, 'degree'].apply(lambda x: re.sub(r"\.", '', x))
df_complex.loc[:, 'degree'] = df_complex.loc[:, 'degree'].apply(lambda x: re.sub(u"’", '', x))

C:\Python27\lib\site-packages\pandas\core\indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [7]:
df_complex.head(20)

,degree,count
2,Bachelor of Arts (BA),77008.0
3,Bachelor of Science (BS),69426.0
4,Bachelor of Science (BS),66456.0
12,Bachelor of Arts (BA),28212.0
15,Bachelor of Business Administration (BBA),19819.0
19,Bachelor of Business Administration (BBA),17468.0
21,Master of Business Administration (MBA),17319.0
24,Master of Business Administration (MBA),13929.0
26,Master of Science (MS),13539.0
27,Master of Science (MS),13196.0


In [8]:
df_complex = df_complex.groupby('degree').sum().sort_values('count', ascending=False).reset_index()

In [9]:
print df_complex.head()
print df_complex.shape

                                      degree     count
0                   Bachelor of Science (BS)  135960.0
1                      Bachelor of Arts (BA)  105264.0
2  Bachelor of Business Administration (BBA)   37313.0
3    Master of Business Administration (MBA)   31258.0
4                     Master of Science (MS)   26754.0
(8621, 2)


In [10]:
df_complex['degree_lower'] = df_complex['degree'].str.lower()

In [11]:
df_complex.head(10)

,degree,count,degree_lower
0,Bachelor of Science (BS),135960.0,bachelor of science (bs)
1,Bachelor of Arts (BA),105264.0,bachelor of arts (ba)
2,Bachelor of Business Administration (BBA),37313.0,bachelor of business administration (bba)
3,Master of Business Administration (MBA),31258.0,master of business administration (mba)
4,Master of Science (MS),26754.0,master of science (ms)
5,Doctor of Philosophy (PhD),14888.0,doctor of philosophy (phd)
6,Associate of Arts (AA),14419.0,associate of arts (aa)
7,Master of Arts (MA),14122.0,master of arts (ma)
8,Associate of Science (AS),13122.0,associate of science (as)
9,Bachelor of Applied Science (BASc),11375.0,bachelor of applied science (basc)


In [12]:
df_complex_lower = df_complex.groupby('degree_lower').sum().sort_values('count', ascending=False).reset_index()

In [13]:
df_detail = pd.merge(df_complex, df_complex_lower, on='degree_lower', suffixes=('_degree', '_degree_lower'))

In [14]:
df_detail.head()

,degree,count_degree,degree_lower,count_degree_lower
0,Bachelor of Science (BS),135960.0,bachelor of science (bs),135970.0
1,Bachelor Of Science (BS),4.0,bachelor of science (bs),135970.0
2,BACHELOR OF SCIENCE (BS),2.0,bachelor of science (bs),135970.0
3,bachelor of science (bs),1.0,bachelor of science (bs),135970.0
4,bachelor of science (BS),1.0,bachelor of science (bs),135970.0


In [15]:
def split_deg_compex(x):
    split_idx = x.index('(')
    return pd.Series([x[:split_idx].strip(), x[split_idx+1:-1].strip()], index=['degree_name', 'degree_acronym'])

In [16]:
df_split = df_detail['degree'].apply(split_deg_compex)

In [17]:
df_split.head()

,degree_name,degree_acronym
0,Bachelor of Science,BS
1,Bachelor Of Science,BS
2,BACHELOR OF SCIENCE,BS
3,bachelor of science,bs
4,bachelor of science,BS


In [20]:
df_full = df_detail.join(df_split)

In [21]:
df_full.to_csv('degree_full.csv', index=False)

In [ ]:
df_full_2 = pd.read_csv('degree_full.csv')

In [ ]:
df_full_2.shape

In [ ]:
df_full_2.drop(['degree_name', 'degree_acronym'], axis=1, inplace=True)

In [ ]:
df_full_2.sort_values('count', ascending=False).head()

In [ ]:
df_full_2 = df_full_2.groupby(df_full_2['degree'].str.lower()).sum().sort_values('count', ascending=False).reset_index()

In [ ]:
df_full_2.head()

In [ ]:
df_full_2.to_csv('degree_full_2.csv', index=False)